In [8]:
import os
import sys
import re
import json
def get_compound(txt):
    compound={}#ddict(str)
    txtlines=txt.split('\n')
    #print(txtlines)
    for idx,item in enumerate( txtlines ):
        if item =="":continue
        raw=item.split(':')
        compound[raw[0]]=':'.join(raw[1:])
        
        if raw[0]== 'Num Peaks':
            if int(raw[1])>0:
                compound['peaks']={}#ddict(float)
                for peak_id in range(idx+1,len(txtlines)):
                    mz_int=txtlines[peak_id].split('\t')
                    compound['peaks'][float(mz_int[0])]=float(mz_int[1])
                break
            else:
                print(f'no peaks in c{compound}')
    return compound

def process_msp(msp_file_path,output_path):
    
    compounds_list=[]
    with open(msp_file_path,'r') as fp:
        fltxt_lines=fp.read().split('\n\n')
    lengh=len(fltxt_lines)
    print(lengh)

    for i, cmp_txt in enumerate( fltxt_lines):
        try:
            cmp=get_compound(cmp_txt)
            compounds_list.append(cmp)
        except Exception as e:
            print(cmp_txt,e)

        #print(f'\r deal the {i}/{lengh} th compound',end='')

    import pandas as pd
    cmp_df=pd.DataFrame(compounds_list)
    print(cmp_df.shape)
    #cmp_df.drop_duplicates(['INCHIKEY'],keep='first',inplace=True)
    #cmp_df.drop_duplicates(['INCHIKEY'],keep='first',inplace=True)
    print(cmp_df.shape)


    #cmp_df.drop_duplicates(['NAME'],keep='first',inplace=True)
    #print(cmp_df.shape)

    cmp_df=cmp_df.reindex()

    temp=cmp_df.dropna(axis=1,how='all')
    print(temp.shape)
    temp=cmp_df.dropna(axis=0,how='all')
    print(temp.shape)
    print(temp.head())

    import pickle

    with open(output_path,'wb' ) as f:
        pickle.dump(temp,f)
        
def generate_lipid_database(lipid_db,output_lipid):
    import pandas as pd
    import pickle
    with open(lipid_db,'rb' ) as f:
        cmp_df=pickle.load(f)
    import re
    import numpy as np

    PC_db=pd.DataFrame(columns=['NAME','PRECURSORMZ','PRECURSORTYPE'])


    patt=re.compile(r'(\d+):(\d+)')
    for idx,content in cmp_df.loc[cmp_df['COMPOUNDCLASS'].isin([" SM"," PC"] ) ,:].iterrows():
        #print(content['NAME'])
        name_txt=content['NAME']

        result=patt.findall(name_txt)
        result=[(int(x[0]),float(x[1])) for x in result ]
        result=np.array(result).sum(axis=0)
        #print(result)
        new_name= f"{content['COMPOUNDCLASS'].replace(' ','')} {int(result[0])}:{int(result[1])}"
        PC_db.loc[len(PC_db),:]=[new_name,content['PRECURSORMZ'],content['PRECURSORTYPE']]
        #break;
    PC_db.drop_duplicates(subset=['NAME','PRECURSORTYPE'],inplace=True)
    PC_db.reset_index(inplace=True)
    print(PC_db)
    with open(output_lipid,'wb') as f:
        pickle.dump(PC_db,f)

In [9]:

#process_msp(r'./MSDIAL-TandemMassSpectralAtlas-VS69-Pos.msp','./lipidMap.picdb')
generate_lipid_database(r'./lipidMap.picdb',r"PC_ms1.db")

      index     NAME  PRECURSORMZ PRECURSORTYPE
0         0  PC 16:0    510.31903        [M+H]+
1         1  PC 17:0    524.33468        [M+H]+
2         2  PC 18:0    538.35033        [M+H]+
3         3  PC 19:0    552.36598        [M+H]+
4         4  PC 20:0    566.38163        [M+H]+
...     ...      ...          ...           ...
3415  47632  SM 81:5   1376.22975       [M+Na]+
3416  47633  SM 81:6   1374.21410       [M+Na]+
3417  47634  SM 81:7   1372.19845       [M+Na]+
3418  47635  SM 81:8   1370.18280       [M+Na]+
3419  47636  SM 81:9   1368.16715       [M+Na]+

[3420 rows x 4 columns]
